In [11]:
import numpy as np
import math 
from svmutil import *

def read_data(dataName):
    yTrain, xTrain = svm_read_problem(dataName)
    return yTrain, xTrain



def to_binary_class(y,label):
    newy = y.copy()
    for i in range(len(y)):
        if newy[i] != label:
            newy[i] = -1
        else:
            newy[i] = 1
    return newy

def sparse_to_dense(xTrain):
    x = xTrain.copy()
    newX = np.zeros(36)
    for index in range(36):
        i = index +1
        if x.get(i) != None:
            newX[index] = x.get(i)
        else:
            newX[index] = 0
    return newX           


In [12]:
#question 15
def calculate_w (yTrain,xTrain):
    yTrain = yTrain.copy()
    xTrain = xTrain.copy()
    m = svm_train(yTrain,xTrain,'-c 10 -t 0')
    w = np.zeros(36)
    for i in range(m.l):
        sv_indice = m.sv_indices[i]-1
        w = w + m.sv_coef[0][i]*sparse_to_dense(xTrain[sv_indice])
        
    print('||w|| = '+str(np.linalg.norm(w)))
    print(m.l)
    return(w)
    
    
        
yTrain,xTrain = read_data('satimage.scale')
newy3 = to_binary_class(yTrain,3)
w = calculate_w (newy3,xTrain)


||w|| = 8.457084298367683
500


In [22]:
 #question 16
    
def kernel(xn,xm):
    xn = xn.copy()
    xm = xm.copy()
    return (1+np.dot(xn,xm))*(1+np.dot(xn,xm))
    
    
def calculate_Ein(y,x):
    y = y.copy()
    x = x.copy()
    m = svm_train(y,x,'-t 1 -d 2 -g 1 -r 1 -c 10')
        
    #b = -m.rho[0];
    #calculate average b
    b = 0
    for k in range(m.l):
        bc = 0
        bc = m.sv_coef[0][k] / abs(m.sv_coef[0][k])
        for i in range(m.l):
            bc = bc - (m.sv_coef[0][i] * kernel(sparse_to_dense(x[m.sv_indices[k]-1]),sparse_to_dense(x[m.sv_indices[i]-1])))
        b = b + bc
        #print(bc)
    b = bc / m.l
    E_in = 0
    for i in range(len(y)):
        gSVM = 0
        for j in range (m.l):
            SVM_index = m.sv_indices[j]-1
            gSVM = gSVM + (m.sv_coef[0][j] * kernel(sparse_to_dense(x[SVM_index]),sparse_to_dense(x[i])) + b)
        if gSVM * y[i] < 0:
            E_in += 1
    E_in = E_in/len(y)
    print ('number of SVM = ' + str(m.l))
    return E_in

In [23]:

newy1 = to_binary_class(yTrain,1)
print ('1 versus not 1 E_in: ' + str(calculate_Ein(newy1,xTrain)))
newy2 = to_binary_class(yTrain,2)
print ('2 versus not 2 E_in: ' + str(calculate_Ein(newy2,xTrain)))
newy3 = to_binary_class(yTrain,3)
print ('3 versus not 3 E_in: ' + str(calculate_Ein(newy3,xTrain)))
newy4 = to_binary_class(yTrain,4)
print ('4 versus not 4 E_in: ' + str(calculate_Ein(newy4,xTrain)))
newy5 = to_binary_class(yTrain,5)
print ('5 versus not 5 E_in: ' + str(calculate_Ein(newy5,xTrain)))


number of SVM = 145
1 versus not 1 E_in: 0.0006764374295377678
number of SVM = 87
2 versus not 2 E_in: 0.0
number of SVM = 433
3 versus not 3 E_in: 0.022322435174746337
number of SVM = 712
4 versus not 4 E_in: 0.040586245772266064
number of SVM = 259
5 versus not 5 E_in: 0.006764374295377677


In [38]:

#question 18
def Gkernel(xn, xm, gamma):
    return math.exp(-gamma * np.linalg.norm(xn - xm) ** 2)

def calculate_Eout(y,x,yTest,xTest,C,gamma):
    y = y.copy()
    x = x.copy()
    
    m = svm_train(y,x,'-t 2 -c ' + str(C) +' -g '+ str(gamma))
    print ('number of SVM = ' + str(m.l))
    #b = -m.rho[0];
    #calculate average b
    k = 0
    b = m.sv_coef[0][k] / abs(m.sv_coef[0][k])
    for i in range(m.l):
        b = b - (m.sv_coef[0][i] * Gkernel(sparse_to_dense(x[m.sv_indices[k]-1]),sparse_to_dense(x[m.sv_indices[i]-1]),gamma))
    E_out = 0
    for i in range(len(yTest)):
        gSVM = 0
        for j in range (m.l):
            SVM_index = m.sv_indices[j]-1
            gSVM = gSVM + (m.sv_coef[0][j] * Gkernel(sparse_to_dense(x[SVM_index]),sparse_to_dense(xTest[i]),gamma) + b)
        if gSVM * yTest[i] < 0:
            E_out += 1
    E_out = E_out/len(yTest)
    return E_out



yTest, xTest = read_data('satimage.scale.t')
newy6 = to_binary_class(yTrain,6)
newYTest = to_binary_class(yTest,6)

print('C = 0.01, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.01,10)))
print('C = 0.1, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,10)))
print('C = 1, Eout = '  + str(calculate_Eout(newy6,xTrain,newYTest,xTest,1,10)))
print('C = 10, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,10,10)))
print('C = 100, Eout = '+ str(calculate_Eout(newy6,xTrain,newYTest,xTest,100,10)))


number of SVM = 3843
C = 0.01, Eout = 0.765
number of SVM = 3883
C = 0.1, Eout = 0.765
number of SVM = 3690
C = 1, Eout = 0.235
number of SVM = 3703
C = 10, Eout = 0.235
number of SVM = 3703
C = 100, Eout = 0.235


In [ ]:
#question 19
print('gamma = 0.1, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,0.1)))
print('gamma = 1, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,1)))
print('gamma = 10, Eout = '  + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,10)))
print('gamma = 100, Eout = ' + str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,100)))
print('gamma = 1000, Eout = '+ str(calculate_Eout(newy6,xTrain,newYTest,xTest,0.1,1000)))

number of SVM = 1102
gamma = 0.01, Eout = 0.235
number of SVM = 1175
gamma = 0.1, Eout = 0.765
number of SVM = 3883
gamma = 1, Eout = 0.765
number of SVM = 4435


In [ ]:
#question 20

